#Entregable1- IRN
###Grupo 3
##Codigo de Generacion de Dataset
###Integrantes:
Diego Vasquez

Fabricio Guerrero


#Codigo de Generación de Dataset


### Primera Parte:
#### En esta parte se instalan e importan las libreías necesarias para el correcto funcionamiento del código, así como también se definen las rutas de archivos de interés.

In [1]:
pip install -r /content/IRN/Entregable1/requirements.txt -q

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/content/IRN/Entregable1/requirements.txt'


In [2]:
import os

from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

from dotenv import load_dotenv, find_dotenv
import PyPDF2
import os
from PIL import Image
import pytesseract
rutaTexto=r"/DataTexto"
pytesseract.pytesseract.tesseract_cmd = R"/content/IRN/Entregable1/tesseract.exe"
load_dotenv(find_dotenv(),override = True)

ModuleNotFoundError: ignored

Transformacion de archivos de PDF Y PNG a TXT

### Segunda parte:
#### Utilización de pytesseract y PyPDF2 para la transformación de archivos .PNG y .PDF a archivos .txt

In [ ]:
#Convierte los archivos PNGs a texto.
def IMGtoText(ruta_img,caso_id):
    imagen= Image.open(ruta_img)
    texto= pytesseract.image_to_string(imagen)

    #Crea la ruta del archivo donde se almacena el texto plano.
    carpeta, nombre_txt = os.path.split(ruta_img)
    nombre_txt = caso_id +"_" + os.path.splitext(nombre_txt)[0] + "_SEGUIMIENTO_EXPEDIENTE.txt"


    ruta_txt = os.path.join(carpeta, nombre_txt)

    with open(ruta_txt, 'w') as archivo:
        archivo.write(texto)

In [ ]:
#Convierte los archivos PDFs a texto.
def PDFtoText(ruta_pdf,caso_id):
    pdf= open(ruta_pdf,"rb")
    reader= PyPDF2.PdfReader(pdf)
    extract_info= ""
    for num in range(len(reader.pages)):
        info_page = reader._get_page(num)
        extract_info= extract_info + info_page.extract_text()

    #Crea la ruta del archivo donde se almacena el texto plano.
    carpeta, nombre_pdf = os.path.split(ruta_pdf)
    nombre_txt = caso_id +"_" + os.path.splitext(nombre_pdf)[0] + ".txt"

    ruta_txt = os.path.join(carpeta, nombre_txt)

    if extract_info != "":
        with open(ruta_txt, 'w') as archivo:
            archivo.write(extract_info)

In [ ]:
def DeterminarID(string):
    indice = 0
    # Elimina los valores iniciales no numéricos del nombre, como 'exp', etc.
    while indice < len(string) and not string[indice].isdigit():
        indice += 1
    string = string[indice:]
    #Dado que el nombre del archivo incluye el estado, se puede obtener el estado de manera deterministica.
    #Este estado deterministico será utilizado solamente para hallar la metrica accuracy
    id_code, estado = string.rsplit('-', 1)
    return id_code,estado

In [ ]:
def ProcesarCarpeta(rutaPrimaria, nombre_carpeta):
    ruta_carpeta_completa = os.path.join(rutaPrimaria, nombre_carpeta)
    lista_archivos = os.listdir(ruta_carpeta_completa)

    for nombre_archivo in lista_archivos:
        ruta_archivo = os.path.join(ruta_carpeta_completa, nombre_archivo)
        caso_id, estado= DeterminarID(nombre_carpeta)
        if ruta_archivo.lower().endswith('.pdf'):
            try:
                PDFtoText(ruta_archivo,caso_id)
                print('Se convirtio PDF')
            except Exception as e:
                print(f"Error al convertir el PDF: {e}")

        elif ruta_archivo.lower().endswith('.png'):
            IMGtoText(ruta_archivo,caso_id)
            print('Se convirtio Imagen')

        else:
            print(f"{nombre_archivo} no es ni PDF ni imagen")

In [ ]:
#En esta celda, se corren las funciones de manera adecuada para lograr obtener los archivos en texto plano
rutaTexto= rutaTexto
lista_carpeta = [nombre for nombre in os.listdir(rutaTexto) if os.path.isdir(os.path.join(rutaTexto, nombre))]
for carpeta in lista_carpeta:
    ruta = os.path.join(rutaTexto,carpeta)
    lista_carpetas = os.listdir(ruta)
    for nombre in lista_carpetas:
        ProcesarCarpeta(ruta, nombre)
        ruta_expediente= os.path.join(ruta,nombre)

    # Imprimir
    print("Lista de nombres de carpetas:")
    print(lista_carpetas)

###Tercera Parte:
#### Se utilizan modelos de LLM, a los cuales se les darán los archivos y ciertos prompts, para obtener el overview y el estado del caso.

In [ ]:
def ia(texto, humanInput):
    llm  = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
    docs = [Document(page_content=texto)]

    template = humanInput + '''
    del siguiente texto:
    TEXTO:{text}.
    '''
    prompt = PromptTemplate(
        input_variables=['text'],
        template=template
    )
    chain = load_summarize_chain(
        llm,
        chain_type='stuff',
        prompt=prompt,
        verbose=False
    )
    resumen = chain.run(docs)
    print(resumen)
    return resumen

Ahora, se probaran distintos promts, ademas de distintas formas de introducir la data.

El primer método de introducir la información es combinando toda la info en un solo string y dándosela de frente a la IA


In [ ]:
def IA1(rutaExpediente,resumenPrompt,estadoPrompt):
    texto_ia= ""
    for archivo in os.listdir(rutaExpediente):
        if archivo.lower().endswith(".txt"):  # Asegúrate de que solo estés leyendo archivos .txt
            ruta_archivo = os.path.join(rutaExpediente, archivo)
            with open(ruta_archivo, 'r') as file:
                contenido = file.read()
                texto_ia += contenido  # Agregar el contenido de cada archivo a la variable de texto combinado

# Imprimir o hacer cualquier operación que necesites con el texto combinado
    overview = ia(texto_ia,resumenPrompt)
    estado= ia(texto_ia,estadoPrompt)
    return overview, estado

El segundo método es darle a la IA archivo por archivo, que saque un resumen por archivo y lo una formando una string, y finalmente sacar un resumen de dicha string unida

In [ ]:

def IA2(rutaExpediente,resumenPrompt,estadoPrompt):

    overview= ""
    estado= ""
    for archivo in os.listdir(rutaExpediente):
        if archivo.lower().endswith(".txt"):  # Asegúrate de que solo estés leyendo archivos .txt
            ruta_archivo = os.path.join(rutaExpediente, archivo)
            with open(ruta_archivo, 'r') as file:
                texto_ia = file.read()
            overview= overview + ia(texto_ia,resumenPrompt)
            estado= estado+ ia(texto_ia,estadoPrompt)
            print("_________________________________________________")


    overview= ia(overview,resumenPrompt)
    estado= ia(estado,estadoPrompt)

El tercer método es un método regresivo, lo que hace es que cada resumen lo introduce nuevamente junto al siguiente archivo no resumido.

In [ ]:
def IA3(rutaExpediente,resumenPrompt,estadoPrompt):

    overview= ""
    estado= ""
    for archivo in os.listdir(rutaExpediente):
        if archivo.lower().endswith(".txt"):  # Asegúrate de que solo estés leyendo archivos .txt
            ruta_archivo = os.path.join(rutaExpediente, archivo)
            with open(ruta_archivo, 'r') as file:
                overview = overview + file.read()
                estado= estado + file.read()
            overview= ia(overview,resumenPrompt)
            estado= ia(estado,estadoPrompt)
            print("_________________________________________________")

    print("Finalizado")

In [ ]:
def Escritura(ruta,caso_id, overview, estado,estado_deterministico):
    #Generamos la Ruta de la carpeta.
    if not os.path.exists(ruta):
        os.makedirs(ruta)
    carpeta = "Archivos_IA_"+caso_id
    ruta_carpeta = os.path.join(ruta, carpeta)
    if not os.path.exists(ruta_carpeta):
        os.makedirs(ruta_carpeta)

    #Generamos la ruta de los archivos.
    ruta_overview = os.path.join(ruta_carpeta,caso_id+ "_overview.txt")
    ruta_estado = os.path.join(ruta_carpeta, caso_id+ "_estado.txt")
    ruta_estado_deterministico = os.path.join(ruta_carpeta, caso_id+ "_estado_deterministico.txt")


    #Crear los archivos
    with open(ruta_overview, 'w') as archivo:
        archivo.write(overview)
    with open(ruta_estado, 'w') as archivo:
        archivo.write(estado)
    with open(ruta_estado_deterministico, 'w') as archivo:
        archivo.write(estado_deterministico)
    print(f"Se han creado el archivos en texto plano.")

### Cuarta Parte:
#### Esta parte nos permite ejecutar las funciones declaradas de manera adecuada, lo que generará el overview y el estado del caso a través de modelos de LLM, de esta manera se logra obtener los archivos pedidos en el apartado B. del entregable 1.
####En este caso, el método final escogido para introducirle data al modelo fue la funcion IA1, debido a que su tiempo de proceso fue muchísimo más corto que el resto de funciones.  

In [ ]:
rutaTexto = rutaTexto
resumenPrompt= "Haz un resumen porfavor"
estadoPrompt= "Determina el estado del caso porfavor"
lista_carpetas= [carpeta for carpeta in os.listdir(rutaTexto) if os.path.isdir(os.path.join(rutaTexto, carpeta))]
for carpeta in lista_carpetas:
    ruta_carpeta= os.path.join(rutaTexto, carpeta)
    lista_expedientes= [expediente for expediente in os.listdir(ruta_carpeta) if os.path.isdir(os.path.join(ruta_carpeta, expediente))]
    for expediente in lista_expedientes:
        ruta_expediente= os.path.join(ruta_carpeta, expediente)
        caso_id, estado_deterministico = DeterminarID(expediente)
        overview,estado = IA1(ruta_expediente,resumenPrompt,estadoPrompt)
        Escritura(ruta_expediente,caso_id, overview, estado,estado_deterministico)

#Código de Limpieza de Data

#####El principal problema en la data, es que hay varios archivos PDF que contienen solamente imagenes, o que son escaneos de documentos en físico. Esto genera que el Código de Generación de Dataset genere archivos tipo .txt totalmente en blanco o en los cuales la mayoría de caracteres son espacios, lo que es indeseable.
Por este motivo se ha realizado el codigo de limpieza, que elimina aquellos archivos cuya longitud sea 0, o si al menos el 50% de sus caracteres son espacios.

In [ ]:
rutaTexto= rutaTexto

lista_carpetas = [nombre for nombre in os.listdir(rutaTexto) if os.path.isdir(os.path.join(rutaTexto, nombre))]

for carpeta in lista_carpetas:
    ruta_carpeta=  os.path.join(rutaTexto,carpeta)
    lista_expedientes= [carpeta for carpeta in os.listdir(ruta_carpeta) if os.path.isdir(os.path.join(ruta_carpeta, carpeta))]

    for expediente in lista_expedientes:
         ruta_expediente = os.path.join(ruta_carpeta,expediente)
         lista_archivos= [nombre for nombre in os.listdir(ruta_expediente)]

         for archivo in lista_archivos:
              ruta_archivo = os.path.join(ruta_expediente,archivo)
              if (ruta_archivo.lower().endswith('.txt')):
                with open(ruta_archivo) as archivo:
                            contenido = archivo.read()
                espacios = sum(1 for c in contenido if c.isspace())
                if len(contenido) == 0:
                    os.remove(ruta_archivo)
                elif (espacios / len(contenido) )>= 0.5 :
                    os.remove(ruta_archivo)





# Código de Métrica Accuracy

In [ ]:
rutaTexto= rutaTexto

lista_carpetas = [nombre for nombre in os.listdir(rutaTexto) if os.path.isdir(os.path.join(rutaTexto, nombre))]
lista_estados= []
lista_estados_deterministicos=[]

for carpeta in lista_carpetas:
    ruta_carpeta=  os.path.join(rutaTexto,carpeta)
    lista_expedientes= [carpeta for carpeta in os.listdir(ruta_carpeta) if os.path.isdir(os.path.join(ruta_carpeta, carpeta))]

    for expediente in lista_expedientes:
         ruta_expediente = os.path.join(ruta_carpeta,expediente)
         lista_IA= [nombre for nombre in os.listdir(ruta_expediente) if os.path.isdir(os.path.join(ruta_expediente, nombre))]

         for nombre in lista_IA:
            ruta_IA=  os.path.join(ruta_expediente,nombre)
            lista_final = [nombre for nombre in os.listdir(ruta_IA) ]

            for nombre in lista_final:
                 ruta_final = os.path.join(ruta_IA,nombre)


                 if ruta_final.lower().endswith('estado.txt'):
                    with open(ruta_final, "r") as archivo:
                            estado= archivo.read()
                    lista_estados.append(estado)
                 elif ruta_final.lower().endswith('estado_deterministico.txt'):
                    with open(ruta_final, "r") as archivo:
                            estado_deterministico= archivo.read()
                    lista_estados_deterministicos.append(estado_deterministico)
print(lista_estados)
print(lista_estados_deterministicos)


In [ ]:
#Esta funcion es para corregir los casos deterministicos en caso haya un error de tipeo, se utiliza una lista con todos los estados posibles del caso
def corregir_estado(estado, estados_validos):
    matches = get_close_matches(estado, estados_validos)
    if matches:
        return matches[0]
    else:
        return estado

estados_validos = [
    "RECHAZADO",
    "INADMISIBLE",
    "EJECUCION",
    "TRAMITE",
    "PLAZO DE IMPUGNACION",
    "CALIFICACION",
    "ARCHIVO DEFINITIVO",
    "ADMITIDA",
    "ATENDIDO",
    "SENTENCIADO",
    "RESUELTO"
]
estados_deterministicos_corregidos = [corregir_estado(estado, estados_validos) for estado in lista_estados_deterministicos]
print(estados_deterministicos_corregidos)
#Ojo: resuelto atendido y sentenciado es el mismo estado, pero con diferente designacion.

In [ ]:
#Esta funcion es para extraer solo el estado de toda la respuesta predicha por la IA, para versiones posteriores se considera utilizar esta funcion para generar el archivo de texto
#En este caso no se utiliza, porque falta informacion acerca de TODOS los estados posibles existentes.
def extraer_estado(cadena, estados_validos):
    for estado in estados_validos:
        if estado.lower()[:-1] in cadena.lower():
            return estado
    return cadena

# Generar la lista de estados predichos2
estados_predichos = [extraer_estado(estado, estados_validos) for estado in lista_estados]
print(estados_predichos)

In [ ]:
#Ahora, ya con los estados extraidos y corregidos, cambiamos normalizamos los elementos resueltos, atendidos y sentenciados, por uno solo:
i=0
for elemento in estados_deterministicos_corregidos:
  if elemento == 'ATENDIDO' or elemento == 'RESUELTO'
    estados_deterministicos_corregidos[i]= 'SENTENCIADO'
  i+=1

i=0
for elemento in estados_predichos:
  if elemento == 'ATENDIDO' or elemento == 'RESUELTO'
    estados_predichos[i]= 'SENTENCIADO'
  i+=1


In [ ]:
#Calculamos la métrica de precisión.
total_predicciones = len(lista_estados)
predicciones_correctas = sum(1 for corr, pred in zip(estados_deterministicos_corregidos,estados_predichos) if corr == pred)
precision = predicciones_correctas / total_predicciones

# Imprimir la precisión
print(f"La precisión del modelo es: {precision:.2%}")

In [ ]:
#En esta parte, calculamos datos más específicos del modelo que podrían ayudarnos a saber cuales son los puntos débiles y fuertes.
estados_validos2= [
    "RECHAZADO",
    "INADMISIBLE",
    "EJECUCION",
    "TRAMITE",
    "PLAZO DE IMPUGNACION",
    "CALIFICACION",
    "ARCHIVO DEFINITIVO",
    "ADMITIDO",
    "SENTENCIADO"
]

# Realizar el análisis para cada estado
for estado in estados_validos2:
    estados_correctos_count = estados_deterministicos_corregidos.count(estado)
    estados_predichos_count = estados_predichos.count(estado)
    estados_correctamente_predichos = sum(1 for predicho in estados_predichos if predicho == estado and predicho in estados_deterministicos_corregidos)
    no_estados_correctamente_predichos = sum(1 for predicho in estados_predichos if predicho == estado and predicho not in estados_deterministicos_corregidos)
    print(f"Para {estado}:")
    print(f"La red neuronal predijo correctamente {estados_correctamente_predichos} estados {estado} de un total de {estados_correctos_count}.")
    print(f"Además, detectó como {estado} {no_estados_correctamente_predichos} casos que no lo eran.")